In [1]:
import tensorflow as tf
from transformers import T5Tokenizer, TFT5ForConditionalGeneration
import pandas as pd
import re
import nltk
import numpy as np
# import pandas as pd
import seaborn as sns
import sklearn as sk
import matplotlib.pyplot as plt

# from nltk.tag import pos_tag
# from nltk.corpus import wordnet
# from nltk.corpus import stopwords
# from nltk.stem import WordNetLemmatizer

# from transformers import BertTokenizer
# from tensorflow.keras.preprocessing.text import Tokenizer
# from tensorflow.keras.preprocessing.sequence import pad_sequences
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Input, Bidirectional
# from tensorflow.keras.callbacks import EarlyStopping
# from tensorflow.keras.callbacks import ReduceLROnPlateau
# from tensorflow.keras.regularizers import l2

# from sklearn.preprocessing import OneHotEncoder
# from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

# nltk downloads and stopwords
# stop_words = stopwords.words("english")
# lemmatizer= WordNetLemmatizer()
# nltk.download('wordnet')#, download_dir = '/root/nltk_data/')
# nltk.download('omw-1.4')#, download_dir = '/root/nltk_data/')
# # !unzip /root/nltk_data/corpora/wordnet.zip -d /root/nltk_data/corpora/
# !unzip /root/nltk_data/corpora/omw-1.4.zip -d /root/nltk_data/corpora/

In [2]:
model_name = "t5-small"  
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = TFT5ForConditionalGeneration.from_pretrained(model_name)


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [3]:
import tensorflow as tf
from transformers import TFT5ForConditionalGeneration

def shift_right(input_ids, pad_token_id, decoder_start_token_id):
    """
    Shifts input ids to the right by one position.
    The first token becomes the decoder_start_token_id.
    """
    batch_size = tf.shape(input_ids)[0]
    # Create a column vector for the start tokens.
    start_tokens = tf.fill([batch_size, 1], decoder_start_token_id)
    # Remove the last token and concatenate the start token at the beginning.
    shifted_input_ids = tf.concat([start_tokens, input_ids[:, :-1]], axis=1)
    return shifted_input_ids

class MultiTaskT5(tf.keras.Model):
    def __init__(self, model_name, num_emotion_labels, num_sentiment_labels):
        super(MultiTaskT5, self).__init__()
        self.t5 = TFT5ForConditionalGeneration.from_pretrained(model_name)
        self.dropout = tf.keras.layers.Dropout(0.1)
        
        # Classification heads for emotion and sentiment.
        self.emotion_classifier = tf.keras.layers.Dense(
            num_emotion_labels,
            activation='softmax',
            name="emotion_classifier"
        )
        self.sentiment_classifier = tf.keras.layers.Dense(
            num_sentiment_labels,
            activation='softmax',
            name="sentiment_classifier"
        )

    def call(self, inputs, task, training=False, labels=None):
        """
        Args:
            inputs: Dictionary with keys 'input_ids' and 'attention_mask'.
            task: One of 'emotion', 'sentiment', or 'summary'.
            training: Boolean flag for training.
            labels: (Optional) target sequence for summarization.
        """
        if task in ['emotion', 'sentiment']:
            # For classification, use only the encoder outputs.
            encoder_outputs = self.t5.encoder(
                input_ids=inputs['input_ids'],
                attention_mask=inputs['attention_mask'],
                training=training
            )
            # Mean-pool the encoder hidden states.
            pooled_output = tf.reduce_mean(encoder_outputs.last_hidden_state, axis=1)
            pooled_output = self.dropout(pooled_output, training=training)
            if task == 'emotion':
                return self.emotion_classifier(pooled_output)
            else:  # task == 'sentiment'
                return self.sentiment_classifier(pooled_output)

        elif task == 'summary':
            if labels is None:
                # For inference, no labels are provided.
                outputs = self.t5(
                    input_ids=inputs['input_ids'],
                    attention_mask=inputs['attention_mask'],
                    training=training
                )
                return outputs.logits
            else:
                # For training, prepare decoder_input_ids by shifting labels to the right.
                pad_token_id = self.t5.config.pad_token_id
                decoder_start_token_id = self.t5.config.decoder_start_token_id
                decoder_input_ids = shift_right(labels, pad_token_id, decoder_start_token_id)
                outputs = self.t5(
                    input_ids=inputs['input_ids'],
                    attention_mask=inputs['attention_mask'],
                    decoder_input_ids=decoder_input_ids,
                    training=training
                )
                return outputs.logits
        else:
            raise ValueError("Unsupported task type. Use 'emotion', 'sentiment', or 'summary'.")

# Example usage:
# Choose a T5 model size appropriate for your resource constraints.
model_name = "t5-small"  
tokenizer = T5Tokenizer.from_pretrained(model_name)

# Instantiate the multi-task model (adjust label numbers as needed).
model = MultiTaskT5(model_name=model_name, num_emotion_labels=6, num_sentiment_labels=3)

# Example input dictionary (normally obtained via tokenizer):
# inputs = tokenizer("summarize: Your input text here", return_tensors="tf")
# For summarization, you might later use model.generate(...) for text generation.


All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [4]:
# Assume emotion task has, for example, 6 labels and sentiment is binary
num_emotion_labels = 6  # Adjust this according to your dataset
num_sentiment_labels = 2


In [5]:

model = MultiTaskT5( model_name, num_emotion_labels, num_sentiment_labels)


All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [6]:
emotion_df = pd.read_csv("/kaggle/input/t5-fine-tune-for-sentiment-detection/train.txt", sep=";", names=["text", "label"], header=None)[:3000]
sentiment_df = pd.read_csv("/kaggle/input/t5-fine-tune-for-sentiment-detection/IMDB Dataset.csv")[:3000]  # Expect columns 'review' and 'sentiment'


In [7]:
from datasets import load_dataset
df_summ = load_dataset("cnn_dailymail", "3.0.0")
df_summ

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

In [8]:
df_train_summ = df_summ['train'][:3000]
df_val_summ   = df_summ['validation'][:300]
df_test_summ  = df_summ['test'][:100]
df_train_summ = pd.concat([pd.Series(df_train_summ['article']),pd.Series(df_train_summ['highlights'])],axis=1).rename(columns={0: 'article', 1: 'highlights'})
df_val_summ = pd.concat([pd.Series(df_val_summ['article']),pd.Series(df_val_summ['highlights'])],axis=1).rename(columns={0: 'article', 1: 'highlights'})
df_test_summ = pd.concat([pd.Series(df_test_summ['article']),pd.Series(df_test_summ['highlights'])],axis=1).rename(columns={0: 'article', 1: 'highlights'})
df_train_summ

,article,highlights
0,"LONDON, England (Reuters) -- Harry Potter star...",Harry Potter star Daniel Radcliffe gets £20M f...
1,Editor's note: In our Behind the Scenes series...,Mentally ill inmates in Miami are housed on th...
2,"MINNEAPOLIS, Minnesota (CNN) -- Drivers who we...","NEW: ""I thought I was going to die,"" driver sa..."
3,WASHINGTON (CNN) -- Doctors removed five small...,"Five small polyps found during procedure; ""non..."
4,(CNN) -- The National Football League has ind...,"NEW: NFL chief, Atlanta Falcons owner critical..."
...,...,...
95,"DENVER, Colorado -- A Colorado man terrorized ...",Some witnesses say Colorado does nothing to pr...
96,"LONDON, England (CNN) -- Previously unseen foo...",NEW: Jury shown new footage of Diana taken hou...
97,WASHINGTON (CNN) -- Republicans reacted with s...,"Republican Sen. Lindsey Graham: ""I am astounde..."
98,"ST. PETERSBURG, Florida (CNN) -- The acrimony ...","YouTube questions address taxes, the Bible, ab..."


In [9]:
def preprocess_texts(texts, max_length=128):
    return tokenizer(texts.tolist(), padding=True, truncation=True,
                     max_length=max_length, return_tensors="tf")


In [10]:
emotion_inputs = preprocess_texts(emotion_df["text"])
# Convert emotion labels (assumed to be strings) to integer codes
emotion_labels = tf.convert_to_tensor(emotion_df["label"].astype('category').cat.codes.tolist())


In [11]:
sentiment_inputs = preprocess_texts(sentiment_df["review"])
# Convert sentiment labels to binary (e.g., "positive" -> 1, "negative" -> 0)
sentiment_labels = tf.convert_to_tensor((sentiment_df["sentiment"]=="positive").astype(int).tolist())


In [12]:
def preprocess_summary_inputs(texts, max_input_length=512):
    """
    Preprocess input texts for summarization by adding the 'summarize: ' prefix
    and tokenizing.
    """
    # Add the summarization prefix to each text
    prefixed_texts = ["summarize: " + text for text in texts.tolist()]
    return tokenizer(prefixed_texts, padding="max_length", truncation=True,
                     max_length=max_input_length, return_tensors="tf")

def preprocess_summary_targets(targets, max_target_length=150):
    """
    Preprocess target summaries by tokenizing.
    """
    # Tokenize the target summaries; we only need the input_ids for targets.
    tokenized_targets = tokenizer(targets.tolist(), padding="max_length", truncation=True,
                                  max_length=max_target_length, return_tensors="tf")
    return tokenized_targets["input_ids"]

# Example usage:
# Assume summary_df is a DataFrame with columns "text" for source texts and "summary" for target summaries.
summary_inputs = preprocess_summary_inputs(df_train_summ["article"])
summary_labels = preprocess_summary_targets(df_train_summ["highlights"])


In [13]:
# Create TensorFlow Datasets
def create_tf_dataset(inputs, labels, batch_size=8):
    dataset = tf.data.Dataset.from_tensor_slices((dict(inputs), labels))
    return dataset.shuffle(1000).batch(batch_size)


In [14]:
emotion_dataset_tf = create_tf_dataset(emotion_inputs, emotion_labels)
sentiment_dataset_tf = create_tf_dataset(sentiment_inputs, sentiment_labels)
summary_dataset_tf = create_tf_dataset(summary_inputs, summary_labels)

In [17]:
# Training setup
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
# optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
# optimizer.build(model.trainable_variables)  # Build optimizer with all model variables
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=3e-5)

In [18]:
epochs = 2
for epoch in range(epochs):
    print(f"Epoch {epoch+1}")

    # Train on emotion task
    for batch in emotion_dataset_tf:
        inputs, labels = batch  # inputs: dict with 'input_ids', 'attention_mask'
        with tf.GradientTape() as tape:
            logits = model(inputs, task="emotion", training=True)
            loss = loss_fn(labels, logits)
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    # Train on sentiment task
    for batch in sentiment_dataset_tf:
        inputs, labels = batch
        with tf.GradientTape() as tape:
            logits = model(inputs, task="sentiment", training=True)
            loss = loss_fn(labels, logits)
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    # Train on summarization task
    for batch in summary_dataset_tf:
        inputs, target_ids = batch  # target_ids: tokenized summary labels
        with tf.GradientTape() as tape:
            # Pass target_ids as labels so that decoder_input_ids are generated internally.
            logits = model(inputs, task="summary", training=True, labels=target_ids)
            # Shift target_ids to compare with the logits.
            # logits: (batch_size, target_seq_length, vocab_size)
            # Use target_ids[:, 1:] as ground truth, and logits[:, :-1, :] for predictions.
            loss = loss_fn(target_ids[:, 1:], logits[:, :-1, :])
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
            
    print(f"Loss after epoch {epoch+1}: {loss.numpy()}")


Epoch 1
Loss after epoch 1: 10.661019325256348
Epoch 2
Loss after epoch 2: 10.52542781829834


In [ ]:
# Save the model weights and tokenizer
model.save_weights("./multi_task_T5_tf_weights")
tokenizer.save_pretrained("./multi_task_T5_tf")


In [19]:
def predict(text, task):
    if task in ['emotion', 'sentiment']:
        # For classification tasks, tokenize with a moderate max_length.
        inputs = tokenizer(text, return_tensors="tf", max_length=128, truncation=True, padding="max_length")
        logits = model(inputs, task=task, training=False)
        # Return the predicted label index.
        return tf.argmax(logits, axis=1).numpy()
    
    elif task == 'summary':
        # Prepend the summarization prefix as required by T5.
        input_text = "summarize: " + text
        # Use a longer max_length for summarization inputs.
        inputs = tokenizer(input_text, return_tensors="tf", max_length=512, truncation=True, padding="max_length")
        # Use the underlying T5 model's generate function.
        generated_ids = model.t5.generate(
            inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_length=150,
            num_beams=4,
            early_stopping=True
        )
        # Decode the generated token ids to text.
        summary = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
        return summary
    
    else:
        raise ValueError("Unsupported task type. Use 'emotion', 'sentiment', or 'summary'.")


In [20]:
# Define mapping dictionaries for sentiment and emotion tasks
sentiment_mapping = {0: "negative", 1: "positive"}
emotion_mapping = {
    0: "anger", 
    1: "fear", 
    2: "joy", 
    3: "love", 
    4: "sadness", 
    5: "surprise"
}

print("Sentiment mapping:", sentiment_mapping)
print("Emotion mapping:", emotion_mapping)

# Function to decode a single sentiment label
def decode_sentiment(label_code):
    return sentiment_mapping.get(label_code, "Unknown")

# Function to decode a single emotion label
def decode_emotion(label_code):
    return emotion_mapping.get(label_code, "Unknown")

# Function to decode the summarization output (which is already a text string)
def decode_summary(summary_text):
    return summary_text

# Optionally, a unified decoding function for all tasks:
def decode_output(prediction, task):
    if task == "sentiment":
        # prediction should be a numeric label code
        return decode_sentiment(int(prediction))
    elif task == "emotion":
        return decode_emotion(int(prediction))
    elif task == "summary":
        # prediction is the generated summary text
        return decode_summary(prediction)
    else:
        return "Unknown task"


Sentiment mapping: {0: 'negative', 1: 'positive'}
Emotion mapping: {0: 'anger', 1: 'fear', 2: 'joy', 3: 'love', 4: 'sadness', 5: 'surprise'}


In [21]:
# Test texts for each task
sentiment_text = "I feel absolutely joyful and excited about the future!"
emotion_text = "This makes me furious!"
summary_text = ("The rapid advances in technology over the past decade have "
                "transformed how we interact with the world, leading to innovative "
                "solutions across industries.")

# Get predictions from the model
sentiment_pred = predict(sentiment_text, task="sentiment")  # Returns a numpy array
emotion_pred = predict(emotion_text, task="emotion")          # Returns a numpy array
summary_pred = predict(summary_text, task="summary")          # Returns a string

# Decode predictions using our mapping functions
print("Sentiment:", decode_sentiment(int(sentiment_pred[0])))
print("Emotion:", decode_emotion(int(emotion_pred[0])))
print("Summary:", summary_pred)


Sentiment: positive
Emotion: joy
Summary: rapid advances in technology over the past decade have transformed how we interact with the world.


In [22]:
# Sample test datasets (lists of raw text) for each task
test_texts_sentiment = [
    "I feel absolutely joyful and excited about the future!",
    "I am really disappointed with the service."
]

test_texts_emotion = [
    "This makes me furious!",
    "I feel so happy and content."
]

test_texts_summary = [
    "The rapid advances in technology over the past decade have transformed the way we live, work, and communicate. These changes have led to groundbreaking innovations across industries.",
    "Artificial intelligence is revolutionizing every sector, from healthcare to finance, by automating processes and providing new insights that were previously unimaginable."
]

print("=== Sentiment Predictions ===")
for text in test_texts_sentiment:
    # Get prediction for sentiment task (expects a numpy array of label indices)
    sentiment_pred = predict(text, task="sentiment")
    # Decode the first (and only) prediction in the batch
    decoded_sentiment = decode_sentiment(int(sentiment_pred[0]))
    print(f"Text: {text}")
    print(f"Predicted Sentiment: {decoded_sentiment}\n")

print("=== Emotion Predictions ===")
for text in test_texts_emotion:
    # Get prediction for emotion task (expects a numpy array of label indices)
    emotion_pred = predict(text, task="emotion")
    # Decode the first prediction in the batch
    decoded_emotion = decode_emotion(int(emotion_pred[0]))
    print(f"Text: {text}")
    print(f"Predicted Emotion: {decoded_emotion}\n")

print("=== Summarization Predictions ===")
for text in test_texts_summary:
    # For summarization, predict returns a generated summary text string.
    summary = predict(text, task="summary")
    print(f"Original Text: {text}")
    print(f"Generated Summary: {summary}\n")


=== Sentiment Predictions ===
Text: I feel absolutely joyful and excited about the future!
Predicted Sentiment: positive

Text: I am really disappointed with the service.
Predicted Sentiment: positive

=== Emotion Predictions ===
Text: This makes me furious!
Predicted Emotion: joy

Text: I feel so happy and content.
Predicted Emotion: joy

=== Summarization Predictions ===
Original Text: The rapid advances in technology over the past decade have transformed the way we live, work, and communicate. These changes have led to groundbreaking innovations across industries.
Generated Summary: rapid advances in technology over the past decade have transformed the way we live, work, and communicate.

Original Text: Artificial intelligence is revolutionizing every sector, from healthcare to finance, by automating processes and providing new insights that were previously unimaginable.
Generated Summary: artificial intelligence is revolutionizing every sector, from healthcare to finance, by automa

In [24]:
# from transformers import T5Tokenizer

# 1. Save T5 encoder-decoder component
model.t5.save_pretrained('multitask_t5')

# 2. Save classification heads (assuming they are grouped into one tf.keras.Model)
tf.keras.models.save_model(model.classification_heads, 'classification_heads')

# 3. Save tokenizer (here using t5-large as the base)
tokenizer = T5Tokenizer.from_pretrained('t5-small')
tokenizer.save_pretrained('saved_t5_tokenizer')


KeyError: 'text'